In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)

    def forward(self, x_i, x_j):
        x = x_i * x_j
        x = self.lin(x)
        return torch.sigmoid(x)

In [14]:
device=0
log_steps=1
num_layers=3
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [15]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [16]:
adj_t = data.adj_t.to(device)

In [17]:
torch.manual_seed(12345)

In [18]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [19]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [20]:
predictor = LinkPredictor(hidden_channels, 1).to(device)

In [21]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [22]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [23]:
model.reset_parameters()
predictor.reset_parameters()

In [24]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [25]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [26]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 1.0219
Epoch: 04, Loss: 0.8787
Epoch: 06, Loss: 0.7773
Epoch: 08, Loss: 0.6717
Epoch: 10, Loss: 0.5953
Epoch: 12, Loss: 0.5372
Epoch: 14, Loss: 0.4937
Epoch: 16, Loss: 0.4577
Epoch: 18, Loss: 0.4256
Epoch: 20, Loss: 0.4003
Epoch: 22, Loss: 0.3760
Epoch: 24, Loss: 0.3564
Epoch: 26, Loss: 0.3413
Epoch: 28, Loss: 0.3248
Epoch: 30, Loss: 0.3103
Epoch: 32, Loss: 0.2982
Epoch: 34, Loss: 0.2875
Epoch: 36, Loss: 0.2787
Epoch: 38, Loss: 0.2702
Epoch: 40, Loss: 0.2615
Epoch: 42, Loss: 0.2547
Epoch: 44, Loss: 0.2488
Epoch: 46, Loss: 0.2409
Epoch: 48, Loss: 0.2345
Epoch: 50, Loss: 0.2300
Epoch: 52, Loss: 0.2253
Epoch: 54, Loss: 0.2195
Epoch: 56, Loss: 0.2157
Epoch: 58, Loss: 0.2129
Epoch: 60, Loss: 0.2062
Epoch: 62, Loss: 0.2018
Epoch: 64, Loss: 0.1985
Epoch: 66, Loss: 0.1962
Epoch: 68, Loss: 0.1921
Epoch: 70, Loss: 0.1882
Epoch: 72, Loss: 0.1847
Epoch: 74, Loss: 0.1826
Epoch: 76, Loss: 0.1815
Epoch: 78, Loss: 0.1771
Epoch: 80, Loss: 0.1743
Epoch: 82, Loss: 0.1734
Epoch: 84, Loss:

In [27]:
predictor.lin.weight

Parameter containing:
tensor([[1.0594, 1.3765, 1.4030, 1.2073, 1.2847, 1.8926, 1.7476, 1.4715, 1.3220,
         1.6448, 1.2935, 1.2769, 1.3668, 1.1633, 1.5616, 1.3923, 1.1169, 1.2621,
         1.8061, 1.1604, 1.5567, 1.3243, 1.2026, 1.2517, 1.4302, 1.1964, 1.3821,
         1.2131, 1.7174, 1.2926, 1.1993, 1.3260, 1.1720, 1.2340, 1.4183, 1.5239,
         1.3956, 1.1428, 1.5941, 1.1481, 1.4405, 1.3382, 2.2938, 1.2132, 1.2269,
         1.7278, 1.0742, 1.1589, 1.1450, 1.6759]], device='cuda:0',
       requires_grad=True)

In [28]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
    (1): SAGEConv(50, 50)
    (2): SAGEConv(50, 50)
  )
)

In [29]:
emb.weight

Parameter containing:
tensor([[ 0.0017,  0.1588,  0.0491,  ..., -0.0741,  0.0020, -0.0931],
        [ 0.0133, -0.0690, -0.2009,  ...,  0.1007,  0.1840,  0.2598],
        [-0.0129, -0.0362, -0.0711,  ..., -0.0292,  0.2554,  0.1769],
        ...,
        [ 0.0126,  0.1991, -0.1394,  ...,  0.1181,  0.0081,  0.0112],
        [ 0.0097,  0.0383,  0.0101,  ...,  0.1161, -0.0635,  0.0791],
        [ 0.0563, -0.0898,  0.0377,  ..., -0.2201, -0.2538, -0.2175]],
       device='cuda:0', requires_grad=True)

In [30]:
h = model(emb.weight, adj_t)

In [31]:
h

tensor([[-0.4772,  0.1681,  0.6466,  ..., -0.7836,  0.6779, -0.0917],
        [ 0.1659, -0.5106, -0.5299,  ...,  0.1456, -0.5962,  0.0016],
        [-0.1346, -0.0655, -0.3014,  ...,  0.0694,  0.0998,  0.1809],
        ...,
        [-0.8629,  0.2061,  0.8775,  ...,  0.3564,  0.3978, -0.6349],
        [-0.3878,  0.1899, -0.0940,  ..., -0.1620, -0.1984,  0.0800],
        [-0.0678, -0.0896,  0.0134,  ...,  0.1909,  0.1918, -0.1371]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [32]:
predictor(h[0], h[411])

tensor([0.9947], device='cuda:0', grad_fn=<SigmoidBackward>)

In [33]:
predictor(h[0], h[2])

tensor([0.0245], device='cuda:0', grad_fn=<SigmoidBackward>)

In [34]:
(h[0] * h[2]).sum()

tensor(0.8425, device='cuda:0', grad_fn=<SumBackward0>)

In [35]:
(h[0] * h[411]).sum()

tensor(7.1340, device='cuda:0', grad_fn=<SumBackward0>)

In [36]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_36_h.npy', h.cpu().detach().numpy())
torch.save(model, '../../models/graphsage_link_pred/36_model.pt')
torch.save(emb, '../../models/graphsage_link_pred/36_embedding.pt')
torch.save(predictor, '../../models/graphsage_link_pred/36_predictor.pt')